## Documentation

# Tasks:
1. Build a EPS database which stores the analysts forecasts and real EPS information for 29 companies 
2. Scraping the data to retrieve the following in the form of tables from the Estimize.com for all available quarters in 2022, 2021, and 2020.
            - Company Basic Information
            - Company Forecasts
            - Analysts Information
            - Stocks Covered
            - Pending Estimates
            - Scored Estimates
            

            


# Company Basic Information

In [ ]:
from selenium import webdriver
import pandas as pd
import csv
import os
from sqlalchemy import create_engine
from selenium import webdriver
import time
import math

In [ ]:
base_url = "https://www.estimize.com/edge"
other_half_url = '?metric_name=eps&chart=historical'

In [ ]:
Ticker_list = ['AMZN', 'AAPL', 'MSFT', 'GOOG', 'TSLA', 'JNJ', 'PG', 'NVDA', 'CSCO', 'BABA', 'HD', 
'BIDU', 'WMT', 'CRM', 'LULU', 'TGT', 'PANW', 'ADBE', 'VMW', 'MU', 'NKE', 'ORCL', 'BB', 'HPQ', 'COST', 'AMAT', 
'BAC', 'CVX', 'AMGN']


for i in range(len(Ticker_list)):
    Ticker_list[i] = Ticker_list[i].lower()

In [ ]:
url = []
for i in range(len(Ticker_list)):
    
    site_name= base_url[:25] + Ticker_list[i] +'/fq1-2020'+ other_half_url
    url.append(site_name)

- Chrome WebDriver is installed and used here in order to run the file.

In [ ]:
driver = webdriver.Chrome()

In [ ]:
ticker = []
name = []
sectors = []
industries = []
followers = []
analysts = []
for i in url:
    driver.implicitly_wait(10)
    driver.get(i)
    driver.implicitly_wait(10)
    ticker.append(driver.find_element_by_class_name("release-header-information-title").text)
    name.append(driver.find_element_by_class_name("release-header-information-description").text)
    sectors.append(driver.find_element_by_xpath('//*[@id="releases_show"]/div[2]/div[2]/div/div[1]/div/div/div/p/span[1]/a/span').text)
    industries.append(driver.find_element_by_xpath('//*[@id="releases_show"]/div[2]/div[2]/div/div[1]/div/div/div/p/span[2]/a/span').text)
    followers.append(driver.find_element_by_xpath('//*[@id="summary-stats"]/div/div/div[1]/div[2]').text)
    analysts.append(driver.find_element_by_xpath('//*[@id="summary-stats"]/div/div/div[2]/a').text)

- The data is scraped using Selenium. 
- Xpath is choosen as as unique identifier for scraping the data of all columns in Company Basic information table 

In [ ]:
dbms_info = pd.DataFrame({"Ticker_list" : ticker, 
    "Company Name" : name,
    "Sectors" : sectors,
    "Industries" : industries,
    "Number Of Followers" : followers,
    "Number Of Analysts" : analysts   
   })

In [ ]:
dbms_info1 = dbms_info.assign(NO_OF_FOLLOWERS = dbms_info['Number Of Followers'].str.replace(",",""), 
                              NO_OF_ANALYSTS = dbms_info['Number Of Analysts'].str.replace(",",""))
dbms_info1.drop(['Number Of Followers', 'Number Of Analysts'], axis =1, inplace = True)
dbms_info1.columns = ['TICKER', 'COMPANY_NAME', 'SECTOR', 'INDUSTRY', 'NO_OF_FOLLOWERS', 'NO_OF_ANALYSTS']
dbms_info2 = dbms_info1.drop_duplicates('TICKER')
dbms_info2.to_sql("company", engine, index = False, if_exists = 'append')

- The Numbumber columns has , in the website. Text Processing and Data cleaning is done in order to run the table in MYSQL.

- The Above Dataframe contains the Basic Information of 29 Companies.
- The columns include:
            1.Ticker_list
            2.Company Name
            3.Sectors
            4.Industries
            5.Number Of Followers
            6.Number Of Analysts

In [ ]:
dbms_info1 = dbms_info.assign(NO_OF_FOLLOWERS = dbms_info['Number Of Followers'].str.replace(",",""), 
                              NO_OF_ANALYSTS = dbms_info['Number Of Analysts'].str.replace(",",""))
dbms_info1.drop(['Number Of Followers', 'Number Of Analysts'], axis =1, inplace = True)
dbms_info1.columns = ['TICKER', 'COMPANY_NAME', 'SECTOR', 'INDUSTRY', 'NO_OF_FOLLOWERS', 'NO_OF_ANALYSTS']
dbms_info2 = dbms_info1.drop_duplicates('TICKER')
dbms_info2.to_sql("company", engine, index = False, if_exists = 'append')

- The above code bring the Dataframe in to MYSQL Databases.

### Company Forecasts

In [ ]:
analyst_type = []
analysts = []
values_list = []
hyperlinks = []
ticker_names = []
quarter_name = []

driver = webdriver.Chrome()
for u in range(len(url)):
    url_i =  url[u]
    driver.implicitly_wait(75)
    driver.get(url_i)
    driver.implicitly_wait(75)
    try:
        num_full = driver.find_element(By.CLASS_NAME, "estimates-tbl-count").text
        index_value = (num_full.find('/'))
        condition = num_full.replace("Showing ","")

        get_num = condition[0:2]
        get_num = pd.to_numeric(get_num)
        num = num_full[index_value+1:].replace("estimates","")
        num = pd.to_numeric(num)
        if num > 30:
            btn = driver.find_element(By.CLASS_NAME, "pagination-footer")
            btn.click()
        driver.implicitly_wait(75)
        
        ticker_i = ticker_copy_list[u]
        tdl = []
        tdl.append(ticker_i)
        ql = []
        ql.append(quarter_list[u])

        # Reading values in one shot
        soup = BeautifulSoup(driver.page_source)
        soup_table = soup.find_all("table")[-1]
        tables = pd.read_html(str(soup_table))
        values = tables[0].drop(["Chart","Unnamed: 1","Rank","Points","Confidence","Last Revised","Analyst"],axis=1)
        values1 = values.Value.to_list()
        
        links = []
        for j in range(0,num):
            links.append(driver.find_elements(By.XPATH,"//a[@class = 'username']")[j]) 
        
        table = soup.find('tbody', attrs={'class':'estimates-tbl-consensus'})
        table_rows  = table.find_all("tr")
        res = []
        for tr in table_rows:
            td = tr.find_all('td')
            row = [tr.text.strip() for tr in td if tr.text.strip()]
            if row:
                res.append(row)
                
        result_consensus = [res[0] for res in res]
        
        res_con = len(result_consensus)
        num_new = num + res_con
        
        ticker_names.extend(tdl*num_new)
        quarter_name.extend(ql*num_new)
        
        analysts.extend(result_consensus)
        analyst_type.extend(['OVERALL']*res_con)
        for i in range(num):
            analysts.append(driver.find_elements(By.CLASS_NAME,"username")[i].text)
        analyst_type.extend(['INDIVIDUAL']*num)
        values_list.extend(values1)
        
        hyperlinks.extend(["NA"]*res_con)
        for link in links:
            hyperlinks.append(link.get_attribute("href"))
        del tables
        del values
        del values1
        driver.implicitly_wait(75)
    except:
        print("Loop Error")

- In order to obtin all the Analys Information, Beautiful Soup and Selenium is used. 
- The values column is Extracted using Beautiful Soup, where as Selenium is used for all the columns data extraction.

In [ ]:
a_p = pd.DataFrame({"Ticker_list" : ticker_names, 
                       "Quarter_year": quarter_name,
                       "Analyst" : analysts,
                       "Value" : values_list,
                       "Analyst_type" : analyst_type,
                       "URL" : hyperlinks
                      })
a_p.columns = ['TICKER', 'QUARTER_YEAR', 'ANALYST', 'VALUE', 'ANALYST_TYPE', 'URL']
a_p.to_sql("company_forecasts", engine, index = False)


- This table contains the following infomation for 29 companies for all available quarters in 2022, 2021, and 2020:
        1. Ticker
        2. Quarter_Year
        3. Analyst
        4. Value
        5. Analyst_Type
        6. URL
        

- The Analyst_Type has 'Overall' and 'Individual' content.
- Here Overall is assigned to Reported Earnings, Estimize Consensus, Estimize Mean and Wall Street Consensus. Individual is assigned to all the individual Analysts.



### Analysts Information

In [ ]:
analyst_urls = pd.read_csv("Analyst_URL.csv")

analyst_urls.drop_duplicates().reset_index(drop = True)

browser = webdriver.Chrome()

name = []
role = []
join = []
score = []
error = []
accuracy = []
points = []
peg = []
stocks = []
pending = []
url_a = []


for url in list(analyst_urls.URL.values):
    browser.get(url)
    browser.implicitly_wait(5)
    
    name.append(browser.find_element(By.XPATH, '//*[@class="before-main-wrapper content-header-show"]/div/div/div/h1/a').text)
    
    role.append(browser.find_element(By.XPATH, '//*[@class="before-main-wrapper content-header-show"]/div/div/div/ul').text)
    
    b= browser.find_element(By.XPATH, '//*[@class="before-main-wrapper content-header-show"]/div/div/div/div[2]').text
    if not b or b == '-':
        join.append('')
    else:
        join.append(b)
    
    m = browser.find_element(By.XPATH, '//*[@class="before-main-wrapper content-header-show"]/div/div[2]/div/div/div[2]').text
    if m == '-' or m == 'N/A':
        score.append('')
    else:
        score.append(float(m))
    
    e = browser.find_element(By.XPATH, '//*[@id="profile-tab-wrap"]/div/div/div').text # int and not existing
    if e  == '-':
        error.append('')
    else:
        error.append(e)
        
    a = browser.find_element(By.XPATH, '//*[@id="profile-tab-wrap"]/div/div[2]/div').text # int and not existing
    if a  == '-':
        accuracy.append('')
    else:
        accuracy.append(a)
        
    p = browser.find_element(By.XPATH, '//*[@id="profile-tab-wrap"]/div[2]/div/div').text # int and not existing
    if p  == '-':
        points.append('')
    else:
        points.append(p)
        
    pe = browser.find_element(By.XPATH, '//*[@id="profile-tab-wrap"]/div[2]/div[2]/div').text # int and not existing
    if pe  == '-':
        peg.append('')
    else:
        peg.append(pe)
        
    s = browser.find_element(By.XPATH, '//*[@id="profile-tab-wrap"]/div[3]/div/div').text # int and not existing
    if s  == '-':
        stocks.append('')
    else:
        stocks.append(s)
        
    pen = browser.find_element(By.XPATH, '//*[@id="profile-tab-wrap"]/div[3]/div/div').text # int and not existing
    if pen  == '-':
        pending.append('')
    else:
        pending.append(pen)
        
    url_a.append(url)

In [ ]:
d8 = {'Name': name, 
      'Roles' :role , 
      'Join Date':join, 
      'Analyst Confidence Score' : score, 
      'Error rate': error,
      'Accuracy Percentile':accuracy, 
      'points' :points, 
      'points/Estimate' : 
      peg, 'stocks' : stocks, 
      'pending':pending, 'URL':url_a}  
ana_ba_info = pd.DataFrame(data=d8)

In [ ]:
ana_ba_info['Join1'] = ana_ba_info['Join Date'].str.replace(".+since\s|\s-.+", "", regex= True)
ana_ba_info['Join2'] = ana_ba_info['Join1'].str.replace("Twitter|StockTwits","", regex=True)
ana_ba_info['Join3'] = ana_ba_info['Join2'].str.replace("\n","", regex=True)

ana_ba_info.drop(['Join1', 'Join2', 'Join Date'], axis=1, inplace =True)

ana_ba_info.columns = ['NAME', 'ROLE', 'ANALYST_CONFIDENCE_SCORE', 'ERROR_RATE', 'ACCURACY_PERCENTILE', 'POINTS', 
                       'POINTS_PER_ESTIMATE', 'STOCKS', 'PENDING', 'URL', 'JOIN_DATE']

ana_ba_info1 = ana_ba_info.drop_duplicates()
ana_ba_info1.to_sql("analyst_info", engine, index = False, if_exists = 'append')

 - This table contains the folllowing columns:
         1. Name
         2. Roles
         3. Join Date
         4. Analyst Confidence Score
         5. Error Rate
         6. Accuracy Percentile
         7. Points
         8. Points/Estimate
         9. Stocks
         10. Pending 
         11. URL
         
         
         
         
- The data is scraped using Selenium. Here the XPath is choosen to scrape the data form each analyst


### Stocks Covered

In [ ]:
driver = webdriver.Chrome()
path = "C:/Users/Rida/Downloads/Stocks/"
lost_url = []
for i in range(50):
    try:
        url_i = analysts_df.iloc[i,0]
        driver.get(url_i)
        driver.execute_script('window.scrollTo(0, 1200);')
        # The above line of code is written to scroll the browser near the Stocks Covered table
        
        driver.implicitly_wait(50)

        num = int(driver.find_elements_by_class_name('profile-tbl-pagination-total-count')[0].text)
        if(num>10):
            driver.implicitly_wait(50)
            for i in range(math.ceil(num/20)):
                btn = driver.find_element_by_xpath('//*[@id="profile-covered-stocks"]/div[2]/a')
                btn.click()
                time.sleep(2) 
        a = driver.find_elements_by_xpath('//*[@id="profile-covered-stocks"]/div[1]/div[2]')[0].text
        
        driver.implicitly_wait(50)

        x = a.replace(" ",",")
        li = list(x.split("\n"))
        Stocks_Covered = pd.DataFrame(li)
        Stocks_Covered[['a', 'b','c','d','e','f','g','h']] = Stocks_Covered[0].str.split(',', expand=True)
        Stocks_Covered["c"] = Stocks_Covered['b'].astype(str) +"-"+ Stocks_Covered["c"].astype(str)
        Stocks_Covered.drop([0,"b"],axis=1,inplace=True)
        Stocks_Covered.columns = ["TICKER","REPORTS","QUARTERS","POINTS","PTS_EST","ERROR RATE","ACCURACY"]
        Stocks_Covered['url'] = url_i
        path_i = path + "Stocks_" + str(i) +".csv"
        Stocks_Covered.to_csv(path_i, index = False)
        del Stocks_Covered
    except:
        lost_url.append(url_i)


In [ ]:
file_path = 'C:/Users/Rida/Downloads/Stocks_Covered/'
import os
files = os.listdir(file_path)
analyst_pred = pd.DataFrame()
for i in range(len(files)):
    print(i)
    path_i = file_path + files[i]
    temp_df = pd.read_csv(path_i)
    analyst_pred= pd.concat([analyst_pred, temp_df])

analyst_pred1 = analyst_pred.assign(ERROR_RATE = analyst_pred['ERROR RATE'].str.replace('%', ""), 
                                    ACCURACY = analyst_pred['ACCURACY'].str.replace('%', ""))
analyst_pred1.drop(['ERROR RATE', 'ACCURACY'], axis = 1, inplace= True)
analyst_pred1.rename(columns = {"url":"URL"}, inplace=True)
analyst_pred1.to_sql("analyst_stocks_covered", engine, index = False)

- This table contains the following:
        1. Ticker
        2. Reports
        3. Quarters
        4. Points
        5. PTS_EST
        6. URL
        7. Error Rate
        
- Due to the constant interruption of 403 Error, we decided to collect the lost URLS and run the code again, in order to retrieve all the information. The information is collected in lost_url list.

- Scrolling the browser till the stocks covered table was required in order to fetch the information. Otherwise it would show "loading".


- The table has **12745** records

### Pending Estimates

In [ ]:
def pe_parallel_func(url_link):
    c = webdriver.ChromeOptions()
    c.add_argument("--incognito")
    
    driver = webdriver.Chrome(options=c)
    driver.implicitly_wait(50)
    driver.get(url_link)
    time.sleep(5)
    driver.execute_script('window.scrollTo(0, 1200);')
    time.sleep(5)    
    try:
        check = driver.find_elements(By.CLASS_NAME,'profile-section-message')[0].text
        if len(check)==0:
            num2 = int(driver.find_elements(By.CLASS_NAME,'profile-tbl-pagination-total-count')[1].text)      
            if (num2 >5):
                  for i in range(math.ceil(num2/20)):
                        btn = driver.find_element(By.XPATH,'//*[@id="profile-pending-estimates"]/div[2]')
                        btn.click()
                        time.sleep(3)

            b = driver.find_elements(By.XPATH,'//*[@id="profile-pending-estimates"]/div[1]/div[2]')[0].text
            print(b)
            b_new = re.sub("[0-9]+\sdays ago","",b)
            x_b = b_new.replace(",","")
            b1 = x_b.split("\n")
            pending_estimates = pd.DataFrame(b1)
            pending_estimates["URL"] = url_link
            path = "C:/Users/Rida/Downloads/dbms/"
            pending_estimates = url_link.replace("https://www.estimize.com/users/", "")
            csv_path = path + 'PE_'+ analyst_name + ".csv"
            pending_estimates.to_csv(csv_path, index =False)
        else:
            print("No Pending estimates")
    except:
        logging.warning(url_link)
        
            
    driver.close()


- Here 'check' is used to see which analyst has pending estimates. 
- If the analyst has no estimates we get the message "No Pending estimates". 

In [ ]:
analyst_ind_df = a_p.loc[a_p.Analyst_type != 'OVERALL', 'URL']
analyst_ind_df1 = analyst_ind_df.drop_duplicates().reset_index(drop = True)
url_links = analyst_ind_df1.to_list()

In [ ]:
for i in range(1,len(url_links)):
    pe_parallel_func(url_links[i])
    time.sleep(2)

- Here Parallel Function is used in order to fetch the data quickly.

In [ ]:
path_3 = "C:/Users/Rida/Downloads/dbms/"
files_2 = os.listdir(path_3)
pending_estimate = pd.DataFrame()
for i in range(len(files_2)):
    print(i)
    path_i = path_3 + files_2[i]
    temp_df = pd.read_csv(path_i)
    pending_estimate= pd.concat([pending_estimate, temp_df])


pending_estimate = pending_estimate.reset_index(drop = True)
pending_estimate['0'] = pending_estimate['0'].replace("\s+", " ", regex = True)
pending_estimate1 = pending_estimate['0'].str.split(' ', expand=True)
pending_estimate1['URL'] = pending_estimate['URL']

pending_estimate1['QUARTER'] = pending_estimate1[1] + " " + pending_estimate1[2]
pending_estimate1['REPORTS'] = pending_estimate1[3] + " " + pending_estimate1[4] + " " + pending_estimate1[5]
                               + " " + pending_estimate1[6]
pending_estimate1['PUBLISHED'] = pending_estimate1[7] + " " + pending_estimate1[8] + " " + pending_estimate1[9]


pending_estimate1.drop([1,2,3,4,5,6,7,8,9], axis = 1, inplace = True)
pending_estimate1.rename(columns = {0:'TICKER', 10:'EPS', 11:'REVENUE'}, inplace = True)
pending_estimate1.to_sql("analyst_pending_estimates", engine, index = False, if_exists='append')

- The DataFrame Pending estimates includes the following information:
       1. TICKER
       2. EPS
       3. Revenue
       4. URL
       5. Quarter
       6. Reports
       7. Published
       
    

- Not every analyst has Pending estimates. Hence this table has **2478** records


### Scored Estimates

In [ ]:
## Scored Estimtes
from selenium import webdriver
import pandas as pd
import csv
import time
import math
import pandas as pd

In [ ]:
analysts_df = pd.read_csv("Analyst_URL.csv")

In [ ]:
driver = webdriver.Chrome()
path = "C:/Users/Rida/Downloads/table_3/"
lost_url = []
for i in range(len(analysts_df)):
    try:
        url_i = analysts_df.iloc[i,0]
        driver.get(url_i)
        print(url_i)
        driver.execute_script('window.scrollTo(0, 1200);')
        time.sleep(10)

        number = int(driver.find_elements_by_class_name('profile-tbl-pagination-total-count')[0].text)        
        if(number>5):
            for i in range(math.ceil(number/20)):
                btn = driver.find_element_by_xpath('//*[@id="profile-scored-estimates"]/div[2]')
                btn.click()
                time.sleep(5)
                continue
        c = driver.find_elements_by_xpath('//*[@id="profile-scored-estimates"]/div[1]/div[2]')[0].text
        time.sleep(10)
        li = list(c.split("\n"))
        scored_estimates = pd.DataFrame(li)
        scored_estimates["URL"] = url_i
        csv_path = path + 'S7_new'+ str(i) + ".csv"
        scored_estimates.to_csv(csv_path, index =False)
        del scored_estimates
    except:
        lost_url.append(url_i) 



In [ ]:
file_path_1 = 'C:/Users/Rida/Downloads/scored_estimates/'
files_1 = os.listdir(file_path_1)
scoring_estimate = pd.DataFrame()
for i in range(len(files_1)):
    print(i)
    path_i = file_path_1 + files_1[i]
    temp_df = pd.read_csv(path_i)
    scoring_estimate= pd.concat([scoring_estimate, temp_df])
scoring_estimate = scoring_estimate.reset_index(drop = True)
scoring_estimate['0'] = scoring_estimate['0'].replace("- DEFUNCT ", "", regex = True)
scoring_estimate1 = scoring_estimate['0'].str.split(' ', expand=True)
scoring_estimate1['URL'] = scoring_estimate['URL']
scoring_estimate1['QUARTER'] = scoring_estimate1[1] + ' ' + scoring_estimate1[2]
scoring_estimate1['REPORTED'] = scoring_estimate1[3] + ' ' + scoring_estimate1[4] + ' ' + scoring_estimate1[5]
scoring_estimate1['RANK'] = scoring_estimate1[6] + ' ' + scoring_estimate1[7] + ' ' + scoring_estimate1[8]
scoring_estimate1.drop([1,2,3,4,5,6,7,8], axis =1, inplace = True)
scoring_estimate1.rename(columns = {0:'TICKER', 9:'EPS_POINTS', 10:'REVENUE_POINTS', 11:'TOTAL_POINTS'}, inplace = True)
scoring_estimate1.to_sql("analyst_scoring_estimate", engine, index = False, if_exists = 'append')

- Data Processing for Scored Estimate Table includes erasing extra test from the main columns and then bifurcating them into columns to obtain a proper table.



- This Scored Estimates table contains the following data:
         1. Ticker
         2. EPS_Points
         3. Revenue_points
         4. Total_Points
         5. URL
         6. Quarter
         7. Reported
         8. Rank
         
         
         
- This table has **113805** Records.

### MYSQL

After scraping the Data, the next task was to convert the Python Dataframes into My_SQL Tables, by using SQLAlchemy.

**1) Given a ticker, how many analysts have made estimations for its EPS? Rank them by their confidence score, total points, error rate or accuracy percentile?**
- Output: 2716 rows for AMZN. 

- Bill_Maurer has the highest score for Q1 2020.

- Output is created for all the quarters.

**2) Given an industry, how many companies are covered, the average number of analysts, the average bias between the Estimize Consensus and the Reported Earnings?**


- Output: 16 rows. 

- Software industry has the highest number of companies covered with average number of analysts of 608

**3) Which company have the largest number of analysts with confidence score greater than 7?**


- Apple has the largest number of analysts with confidence score greater than 7

**4) Who has the largest number of followers?**
- Apple has largest number of followers.

### Regression Model (BONUS QUESTION)

**Objective:** To identify the error rate of an analyst with respect to different scores of an analyst.

Independent Variables:​

1. Points​

2. Accuracy Percentile​

3. Points per estimate​

4. Stocks.

Target Variable:​

1. Error rate

In [ ]:
#Importing the files
import pandas as pd
import numpy as np
import sqlalchemy
from sqlalchemy import create_engine

In [ ]:
user = 'root'
password = 'root'
host = 'localhost'
port = 3306
database = 'estimize'

engine = create_engine(url="mysql+pymysql://{0}:{1}@{2}:{3}/{4}".format(user, password, host, port, database))

In [ ]:
a= pd.read_sql("Select * from estimize.analyst_info", engine)
b = a.dropna()

In [ ]:
b.ERROR_RATE = np.log((b["ERROR_RATE"]))
b.POINTS = np.sqrt(b["POINTS"])
b.ACCURACY_PERCENTILE =  np.sqrt((b["ACCURACY_PERCENTILE"]))
b.POINTS_PER_ESTIMATE = np.sqrt(b["POINTS_PER_ESTIMATE"])
b.POINTS_PER_ESTIMATE = np.sqrt(b["STOCKS"])

In [ ]:
b = b.dropna()

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
data1 = scaler.fit_transform(b[["ERROR_RATE", "POINTS", "ACCURACY_PERCENTILE" ,"POINTS_PER_ESTIMATE", "STOCKS"]].to_numpy())

In [ ]:
new_data = pd.DataFrame(data1, columns = ["ERROR_RATE", "POINTS", "ACCURACY_PERCENTILE" ,"POINTS_PER_ESTIMATE", "STOCKS"])

In [ ]:
new_data.corr()

In [ ]:
plt.style.use("seaborn")
new_data.hist(figsize = (25,20))

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize=(16, 6))
heatmap = sns.heatmap(new_data.corr(), vmin=-1, vmax=1, annot=True)
heatmap.set_title('Correlation Heatmap', fontdict={'fontsize':12}, pad=12);

### Training and testing the data

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test,y_train, y_test = train_test_split(new_data.drop("ERROR_RATE", axis=1),
                                                  new_data["ERROR_RATE"],
                                                  test_size = 0.3,
                                                  random_state = 0)

X_train.shape, y_train.shape

### LINEAR REGRESSION

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error,mean_absolute_error
linreg = LinearRegression()
reg = linreg.fit(X_train, y_train)
reg.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import r2_score
y_pred = linreg.predict(X_test)
final_score = r2_score(y_test,y_pred)

In [ ]:
final_score

In [ ]:
MSE = mean_squared_error(y_test, y_pred)
MSE

## Random Forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
clf=RandomForestRegressor(n_estimators=100)
clf.fit(X_train,y_train)

y_pred=clf.predict(X_test)
final_score = r2_score(y_test,y_pred)

In [ ]:
final_score

In [ ]:
MSE = mean_squared_error(y_test, y_pred)
MSE

### LGBM Regressor

In [ ]:
import lightgbm as ltb
ltb = ltb.LGBMRegressor()
ltb.fit(X_train, y_train)

In [ ]:
y_pred=ltb.predict(X_test)
final_score = r2_score(y_test,y_pred)

In [ ]:
final_score

In [ ]:
MSE = mean_squared_error(y_test, y_pred)
MSE

### XGB Regressor

In [ ]:
from xgboost import XGBRegressor
xgboost = XGBRegressor(n_estimators=100, max_depth=10, eta=0.1, subsample=0.7, colsample_bytree=0.8)
xgboost.fit(X_train, y_train)

In [ ]:
y_pred=xgboost.predict(X_test)
final_score = r2_score(y_test,y_pred)

In [ ]:
final_score

In [ ]:
MSE = mean_squared_error(y_test, y_pred)
MSE